# Testing the implementation

In [ ]:
import os
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from models import *
from trainer import *
from preprocess import *
from callbacks import *

## 0 Data

In [ ]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/processed_data.csv'))
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'optionid', 'securityid'], axis=1)
df.sample(5)

Create the first neueral network that uses the same set of inputs as the Black-Scholes model

In [ ]:
dataframe_BS = np.vstack((df['strike'].values,
                      df['asset_price'].values,
                      df['days_to_maturity'].values,
                      df['volatility'].values,
                      df['rate'].values,
                      df['contract_price'].values)).T                

In [ ]:
train_ds, valid_ds, test_ds = pipeline1(dataframe_BS, scaling=False)

train_copy, valid_copy, test_copy = pipeline1(dataframe_BS, prefetch=False)

## 1 Start training

Define some parameters

In [ ]:
print_num_epochs = 5 # print progress every print_num_epochs epochs

path_to_save = os.path.join(parent_path, 'NeuralNetwork/models/')  # path to save the model

patience = 10  

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

loss = tf.keras.losses.MeanAbsoluteError(name='loss')

metrics = tf.keras.metrics.MeanAbsolutePercentageError(name='accuracy')

num_epochs = 10  

model = getModel(input_shape = (5, ),
                 hidden_units = [14, 14, 14],
                 output_shape = (1, ),
                 batchnorm = True)
model.summary()

In [ ]:
# define some callbacks  
ckpt = CheckpointCallback(path_to_save)
printing =PrintProgress(num_epochs=print_num_epochs)
early_stop = tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss')
callbacks = [ckpt, printing, early_stop]

In [ ]:
history = compile_and_fit(model,
                          optimizer,
                          loss,
                          num_epochs,
                          train_ds,
                          valid_ds,
                          metrics,
                          callbacks
                          )